In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('/content/complete_solar_data2.csv')


In [5]:
df

,DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,AC_POWER,DAILY_YIELD
0,5/15/2020 0:00,27.004764,25.060789,0.0,0.0,9425.000000
1,5/15/2020 0:00,27.004764,25.060789,0.0,0.0,0.000000
2,5/15/2020 0:00,27.004764,25.060789,0.0,0.0,3075.333333
3,5/15/2020 0:00,27.004764,25.060789,0.0,0.0,269.933333
4,5/15/2020 0:00,27.004764,25.060789,0.0,0.0,3177.000000
...,...,...,...,...,...,...
67693,6/17/2020 23:45,23.202871,22.535908,0.0,0.0,4157.000000
67694,6/17/2020 23:45,23.202871,22.535908,0.0,0.0,3931.000000
67695,6/17/2020 23:45,23.202871,22.535908,0.0,0.0,4322.000000
67696,6/17/2020 23:45,23.202871,22.535908,0.0,0.0,4218.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67698 entries, 0 to 67697
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DATE_TIME            67698 non-null  object 
 1   AMBIENT_TEMPERATURE  67698 non-null  float64
 2   MODULE_TEMPERATURE   67698 non-null  float64
 3   IRRADIATION          67698 non-null  float64
 4   AC_POWER             67698 non-null  float64
 5   DAILY_YIELD          67698 non-null  float64
dtypes: float64(5), object(1)
memory usage: 3.1+ MB


In [8]:
df.describe()

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,AC_POWER,DAILY_YIELD
count,67698.000000,67698.000000,67698.000000,67698.000000,67698.000000
mean,27.986756,32.607233,0.229204,241.277825,3294.890295
std,4.021294,11.226446,0.309365,362.112118,2919.448386
min,20.942385,20.265123,0.000000,0.000000,0.000000
25%,24.570349,23.685627,0.000000,0.000000,272.750000
50%,26.910352,27.433723,0.018554,0.000000,2911.000000
75%,30.912601,40.019036,0.431027,438.215000,5534.000000
max,39.181638,66.635953,1.098766,1385.420000,9873.000000


In [9]:
df.columns

Index(['DATE_TIME', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION',
       'AC_POWER', 'DAILY_YIELD'],
      dtype='object')

In [10]:
df

,DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,AC_POWER,DAILY_YIELD
0,5/15/2020 0:00,27.004764,25.060789,0.0,0.0,9425.000000
1,5/15/2020 0:00,27.004764,25.060789,0.0,0.0,0.000000
2,5/15/2020 0:00,27.004764,25.060789,0.0,0.0,3075.333333
3,5/15/2020 0:00,27.004764,25.060789,0.0,0.0,269.933333
4,5/15/2020 0:00,27.004764,25.060789,0.0,0.0,3177.000000
...,...,...,...,...,...,...
67693,6/17/2020 23:45,23.202871,22.535908,0.0,0.0,4157.000000
67694,6/17/2020 23:45,23.202871,22.535908,0.0,0.0,3931.000000
67695,6/17/2020 23:45,23.202871,22.535908,0.0,0.0,4322.000000
67696,6/17/2020 23:45,23.202871,22.535908,0.0,0.0,4218.000000


In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv("/content/complete_solar_data2.csv")

# Convert DATE_TIME to datetime format
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])

# Extract time-based features
df['Hour'] = df['DATE_TIME'].dt.hour
df['Day'] = df['DATE_TIME'].dt.day
df['Month'] = df['DATE_TIME'].dt.month
df['Weekday'] = df['DATE_TIME'].dt.weekday

# Drop the original DATE_TIME column
df.drop(columns=['DATE_TIME'], inplace=True)

# Handling missing values
df.fillna(df.median(), inplace=True)  # Fill missing values with median

# Define features and target variable
X = df.drop(columns=['AC_POWER'])  # Features (all columns except AC_POWER)
y = df['AC_POWER']  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train an XGBoost model
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_reg.fit(X_train, y_train)

# Predictions
y_pred = xgb_reg.predict(X_test)

# Evaluate model performance
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}, R²: {r2:.4f}")


MAE: 37.24, MSE: 11720.65, RMSE: 108.26, R²: 0.9101
